In [2]:
import pandas as pd
from pathlib import Path
RAW_PATH = Path("../data/raw")

df = pd.concat(
    [pd.read_parquet(p) for p in RAW_PATH.glob("year=*/*.parquet")],
    ignore_index=True
)

df['calendar_date'] = pd.to_datetime(df['calendar_date'])
df['cases'] = df['cases'].astype(float)

print("Data loaded successfully")
df_2020 = df[df['calendar_date'].dt.year == 2020]

print("Total rows in 2020:", len(df_2020))
warehouse_cases_2020 = (
    df_2020
    .groupby("warehouse_id")["cases"]
    .sum()
    .reset_index()
)

warehouse_cases_2020.head()


Data loaded successfully
Total rows in 2020: 7695


,warehouse_id,cases
0,6005,6367500.0
1,6014,12685467.0
2,6022,10998979.0
3,6029,11277863.0
4,6041,9781036.0


In [3]:
# Convert to list of tuples
data = list(warehouse_cases_2020.itertuples(index=False, name=None))

def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])

    return merge(left, right)


def merge(left, right):
    sorted_list = []
    i = j = 0

    while i < len(left) and j < len(right):
        # Compare based on total_cases (index 1)
        if left[i][1] <= right[j][1]:
            sorted_list.append(left[i])
            i += 1
        else:
            sorted_list.append(right[j])
            j += 1

    # Append remaining elements
    sorted_list.extend(left[i:])
    sorted_list.extend(right[j:])

    return sorted_list


In [4]:
sorted_data = merge_sort(data)
sorted_warehouse_2020 = pd.DataFrame(
    sorted_data,
    columns=["warehouse_id", "total_cases_2020"]
)

sorted_warehouse_2020


,warehouse_id,total_cases_2020
0,6005,6367500.0
1,6041,9781036.0
2,7842,10056999.0
3,6022,10998979.0
4,6029,11277863.0
5,6014,12685467.0


In [9]:
import time
import copy
comparison_count = 0  # global counter

def merge_sort_count(arr):
    global comparison_count
    
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = merge_sort_count(arr[:mid])
    right = merge_sort_count(arr[mid:])

    return merge_count(left, right)


def merge_count(left, right):
    global comparison_count
    
    sorted_list = []
    i = j = 0

    while i < len(left) and j < len(right):
        comparison_count += 1   # count comparison
        
        if left[i][1] <= right[j][1]:
            sorted_list.append(left[i])
            i += 1
        else:
            sorted_list.append(right[j])
            j += 1

    sorted_list.extend(left[i:])
    sorted_list.extend(right[j:])
    return sorted_list


In [10]:
comparison_count = 0
sorted_data = merge_sort_count(copy.deepcopy(data))

print("Total comparisons (Merge Sort):", comparison_count)

data_copy = copy.deepcopy(data)

start_time = time.time()
merge_sort_count(data_copy)
end_time = time.time()

print("Execution time (seconds):", end_time - start_time)


Total comparisons (Merge Sort): 10
Execution time (seconds): 0.0
